# Signal Matching Analysis for mannual annotations

In [1]:
from time import gmtime, strftime
import numpy as np

import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import cv2 as cv
import time

import skimage
from at_synapse_detection import SynapseDetection as syn
from PIL import Image, ImageSequence
from skimage import measure
from tqdm import tqdm
from scipy import ndimage
import imageio
from skimage import io
import skimage.io as skio

## Load annotations

In [2]:
def loaddata(fn): 
    syn_vol = skio.imread(fn, plugin='tifffile')
    syn_vol = np.transpose(syn_vol, (1, 2, 0))

    return syn_vol

In [3]:
normalized_receptor_filelist = \
['../notebooks/KDM-SYN-201228/ROI2_data/ROI2-PSD95-P.tif',\
 '../notebooks/KDM-SYN-201228/ROI2_data/ROI2-Synapsin12-P.tif',\
 '../notebooks/KDM-SYN-201228/ROI2_data/ROI2-Gephyrin-P.tif',\
 '../notebooks/KDM-SYN-201228/ROI2_data/ROI2-GABA-P.tif',\
 '../notebooks/KDM-SYN-201228/ROI2_data/ROI2-GephyrinMinusGABA-P.tif',\
 '../notebooks/KDM-SYN-201228/ROI2_data/ROI2-GluA1-P.tif',\
 '../notebooks/KDM-SYN-201228/ROI2_data/ROI2-GluA2-P.tif',\
 '../notebooks/KDM-SYN-201228/ROI2_data/ROI2-GluA3-P.tif', \
 '../notebooks/KDM-SYN-201228/ROI2_data/ROI2-GluA4-P.tif',\
 '../notebooks/KDM-SYN-201228/ROI2_data/ROI2-GluN1-P.tif',\
 '../notebooks/KDM-SYN-201228/ROI2_data/ROI2-GluN2B-P.tif',\
 '../notebooks/KDM-SYN-201228/ROI2_data/ROI2-VGluT1-P.tif' ]

channel_list = \
['PSD95',\
 'Synapsin12',\
 'Gephyrin',\
 'GABA',\
 'GephyrinMinusGABA',\
 'GluA1',\
 'GluA2',\
 'GluA3', \
 'GluA4',\
 'GluN1',\
 'GluN2B',\
 'VGluT1']




In [4]:
annotation_data = io.imread('../KDM-SYN-201228/ROI2_data/AnnotationsExcSynapsesROI2-32b.tif', plugin='tifffile')
annotation_data = np.transpose(annotation_data, (1, 2, 0))

In [5]:
idlist = []
for n in range(0, annotation_data.shape[2]): 
    idlist.append(np.unique(annotation_data[:, :, n]))

In [6]:
idlist = np.unique(np.concatenate(idlist))


In [7]:
len(idlist)

551

In [8]:
474

474

In [9]:
columns_list = [] 
for n_fn in range(0, len(normalized_receptor_filelist)): 

    channel_name = channel_list[n_fn]
    ch_label = channel_name + '_label'
    intensity_sum = channel_name + '_intensity_sum'
    intensity_avg = channel_name + '_intensity_avg'
    ch_slices = channel_name + '_slices'
    ch_pixels = channel_name + '_pixels'

    columns_list.append(intensity_sum) 
    columns_list.append(intensity_avg) 
    columns_list.append(ch_pixels) 

In [10]:
idlist = list(map(int, idlist))

In [11]:
import time



In [12]:
t0 = time.time()
f=2
t1 = time.time()

total = t1-t0
print(total) 

5.7697296142578125e-05


In [13]:
df = pd.DataFrame(columns=columns_list, index=idlist)
annotation_data = io.imread('../KDM-SYN-201228/ROI2_data/AnnotationsExcSynapsesROI2-32b.tif', plugin='tifffile')
annotation_data = np.transpose(annotation_data, (1, 2, 0))

for n_slice in tqdm(range(0, annotation_data.shape[2])): 

    result_props = measure.regionprops(np.int64(annotation_data[:, :, n_slice]), annotation_data[:, :, n_slice])

    for n_anno in range(0, len(result_props)): 
        annoitr = result_props[n_anno].intensity_mean
        anno_slice = annotation_data[:, :, n_slice]==annoitr
        anno_slice = np.uint8(anno_slice>0)
        SE = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(108, 108))
        mask = cv2.dilate(anno_slice, SE)

        for n_fn in range(0, len(normalized_receptor_filelist)): 
            fn = normalized_receptor_filelist[n_fn]
            channel_name = channel_list[n_fn]
            
            intensity_sum = channel_name + '_intensity_sum'
            intensity_avg = channel_name + '_intensity_avg'
            ch_pixels = channel_name + '_pixels'
            
            data = loaddata(normalized_receptor_filelist[n_fn])
            data_slice = data[:, :, n_slice]
            output = data_slice*mask
    
            summed_intensity = np.sum(output)
            number_of_pixels = np.sum(mask)
            avg_intensity = summed_intensity/number_of_pixels
            annoitr = annoitr.astype(np.int32)

            if pd.isnull(df.loc[annoitr, intensity_sum]): 
                df.loc[annoitr, intensity_sum] = summed_intensity
                df.loc[annoitr, ch_pixels] = number_of_pixels
                df.loc[annoitr, intensity_avg] = avg_intensity

            else: 
                df.loc[annoitr, intensity_sum] = df.loc[annoitr, intensity_sum] + summed_intensity
                df.loc[annoitr, ch_pixels] = df.loc[annoitr, ch_pixels] + number_of_pixels
                df.loc[annoitr, intensity_avg] = df.loc[annoitr, intensity_sum] / df.loc[annoitr, ch_pixels]

df.to_csv('sig_extract_ROI2_JULY5_2024.csv')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [3:02:58<00:00, 243.98s/it]
